In [2]:
from flask import Flask, render_template
from flask_sqlalchemy import SQLAlchemy
import requests
from bs4 import BeautifulSoup
import pandas as pd

app = Flask(__name__)

# 設定 SQLAlchemy 與資料庫的連接
app.config['SQLALCHEMY_DATABASE_URI'] = 'mssql+pyodbc:///?' \
                                        'driver=ODBC+Driver+17+for+SQL+Server&' \
                                        'trusted_connection=yes&' \
                                        'server=WEIWEI&' \
                                        'database=master&' \
                                        'UID=wei&' \
                                        'PWD=132476&' \
                                        'TrustServerCertificate=yes'

db = SQLAlchemy(app)

# 定義 'weather' 資料表的模型
class Weather(db.Model):
    __tablename__ = 'weather'
    sid = db.Column(db.Integer, primary_key=True)
    date = db.Column(db.String(50), nullable=False)
    time = db.Column(db.String(100))
    temperature = db.Column(db.String(50))
    humidity = db.Column(db.String(50))
    direction = db.Column(db.String(50))
    speed = db.Column(db.String(50))
    gust = db.Column(db.String(50))
    visibility = db.Column(db.String(50))
    pressure = db.Column(db.String(50))

    def __init__(self, date, time, temperature, humidity, direction, speed, gust, visibility, pressure):
        self.date = date
        self.time = time
        self.temperature = temperature
        self.humidity = humidity
        self.direction = direction
        self.speed = speed
        self.gust = gust
        self.visibility = visibility
        self.pressure = pressure

# 定義抓取天氣資料的函數
def scrape_weather_data():
    # 取得網頁內容
    resp = requests.get("https://www.cwa.gov.tw/V8/C/W/Observe/MOD/24hr/46692.html")
    soup = BeautifulSoup(resp.text, 'html.parser')

    # 初始化資料列表
    data = []
    # 遍歷表格行
    for row in soup.find_all("tr"):
        row_data = list(row.stripped_strings)
        # 只加入符合預期資料長度的行
        if len(row_data) == 14:
            data.append([
                row_data[0],  # 日期
                row_data[1],  # 時間
                row_data[2],  # 溫度
                row_data[3],  # 濕度
                row_data[4],  # 風向
                row_data[5],  # 風速
                row_data[6],  # 陣風
                row_data[10], # 能見度
                row_data[11], # 海平面氣壓
            ])

    # 定義欄位名稱
    columns = ["日期", "時間", "溫度", "濕度", "風向", "風速", "陣風", "能見度", "海平面氣壓"]

    # 建立 DataFrame
    df = pd.DataFrame(data, columns=columns)

    return df

# 定義儲存資料至資料庫的函數
def save_to_database(data, app, db):
    with app.app_context():
        db.create_all()  # 創建所有定義的資料表
        for _, row in data.iterrows():
            # 創建 Weather 物件並將其添加至資料庫會話
            weather_entry = Weather(
                date=row['日期'],
                time=row['時間'],
                temperature=row['溫度'],
                humidity=row['濕度'],
                direction=row['風向'],
                speed=row['風速'],
                gust=row['陣風'],
                visibility=row['能見度'],
                pressure=row['海平面氣壓']
            )
            db.session.add(weather_entry)
        db.session.commit()  # 提交會話

# 定義將資料庫中的資料匯出到 Excel 檔案的函數
def export_to_excel():
    with app.app_context():
        weathers = Weather.query.all()
        data = [{
            '日期': weather.date,
            '時間': weather.time,
            '溫度': weather.temperature,
            '濕度': weather.humidity,
            '風向': weather.direction,
            '風速': weather.speed,
            '陣風': weather.gust,
            '能見度': weather.visibility,
            '海平面氣壓': weather.pressure
        } for weather in weathers]

        df = pd.DataFrame(data)
        file_path = r"C:\Users\Willi\OneDrive\Desktop\新增資料夾 (2)\weather_data_exported.xlsx"
        df.to_excel(file_path, index=False)
        return df

# 定義顯示網頁的路由
@app.route('/')
def index():
    df = export_to_excel()
    return render_template('index.html', tables=[df.to_html(classes='data')], titles=df.columns.values)

if __name__ == '__main__':
    df = scrape_weather_data()
    save_to_database(df, app, db)
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [02/Jul/2024 03:03:30] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jul/2024 03:03:30] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [02/Jul/2024 03:04:10] "GET / HTTP/1.1" 200 -
